In [1]:
from headers import *

In [2]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import numpy as np

## 0. input preparation

In [3]:
df_analysis = pd.read_csv('RTCI_data.csv', index_col = 'index')

In [4]:
df_analysis

,pass_ID,scenario,time_crit,JRT,WT,commuter,age_50_65,age_65plus,Crowding_3_2,Crowding_4_3,Crowding_4_2,output_choice
index,,,,,,,,,,,,
0,1,1,1,20,10,0,0,1,1,0,0,0
0,1,2,1,20,5,0,0,1,1,0,0,1
0,1,3,1,20,10,0,0,1,0,1,0,0
0,1,4,1,20,5,0,0,1,0,1,0,1
0,1,5,1,20,10,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
373,374,2,0,8,5,1,0,0,1,0,0,1
373,374,3,0,8,10,1,0,0,0,1,0,1
373,374,4,0,8,5,1,0,0,0,1,0,1


## DF attributes:
### [pass_id]
### [scenario]
### [time-crit] - 1 if need to arrive on-time
### [JRT] - journey time in [mins]
### [WT] - waiting time in [mins]
### [commuter]
### [age_50-65]
### [age_65plus]
### [Crowding_3_2] - RTCI ratio between 1st and 2nd dep. - scenarios 1-2 (1: 10-minute wait, 2: 5-minute wait)
### [Crowding_4_3] - RTCI ratio between 1st and 2nd dep. - scenarios 3-4 (3: 10-minute wait, 4: 5-minute wait)
### [Crowding_4_2] - RTCI ratio between 1st and 2nd dep. - scenarios 5-6 (5: 10-minute wait, 6: 5-minute wait)
### [output_choice] - 1 if [WAITS] for 2nd dep. / 0 if takes 1st dep. [NOW]

In [5]:
df_analysis.describe()

,pass_ID,scenario,time_crit,JRT,WT,commuter,age_50_65,age_65plus,Crowding_3_2,Crowding_4_3,Crowding_4_2,output_choice
count,2244.000000,2244.000000,2244.000000,2244.000000,2244.000000,2244.000000,2244.000000,2244.000000,2244.000000,2244.000000,2244.000000,2244.000000
mean,187.500000,3.500000,0.443850,21.885027,7.500000,0.791444,0.048128,0.093583,0.333333,0.333333,0.333333,0.462567
std,107.988179,1.708206,0.496948,11.847228,2.500557,0.406367,0.214085,0.291312,0.471510,0.471510,0.471510,0.498708
min,1.000000,1.000000,0.000000,2.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,94.000000,2.000000,0.000000,15.000000,5.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,187.500000,3.500000,0.000000,20.000000,7.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,281.000000,5.000000,1.000000,30.000000,10.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,374.000000,6.000000,1.000000,90.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# BIOGEME MODELS

# 1. general MNL

### Utility (dep1) = 0 (fixed reference value)
### Utility (dep2) = booleans (s1, s2...) + travel times (wt, jrt)
### Utility (dep2) = = [B_crowding_3-2] * s1 + [B_crowding_4-3] * s2 + [B_timecrit] * s3+ [B_commuter] * s4 + [B_age_50-65] * s5 + [B_age_65plus] * s6
###                        + [B_wt] * WT + [B_jrt] * JRT

In [6]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_analysis)

In [7]:
# parameters to be estimated
ASC_FIRST = Beta('ASC_FIRST',0,None,None,1)
BETA_timecrit = Beta('BETA_timecrit',0,None,None,0)
BETA_CR32 = Beta('BETA_CR32',0,None,None,0)
BETA_CR43 = Beta('BETA_CR43',0,None,None,0)
BETA_CR42 = Beta('BETA_CR42',0,None,None,0)
BETA_commuter = Beta('BETA_commuter',0, None,None,0)
BETA_AGE_50_65 = Beta('BETA_AGE_50_65',0,None,None,0)
BETA_AGE_65 = Beta('BETA_AGE_65',0,None,None,0)
BETA_IVT = Beta('BETA_IVT',0,None,None,0)
BETA_WT = Beta('BETA_WT',0,None,None,0)

# variables
s1 = Crowding_3_2 
s2 = Crowding_4_3
s3 = Crowding_4_2
s4 = time_crit
s5 = commuter
s6 = age_50_65
s7 = age_65plus
ivt = JRT 
wt = WT 

# utilities
V1 = ASC_FIRST

V2 = BETA_CR32 * s1 + \
     BETA_CR43 * s2 + \
     BETA_CR42 * s3 + \
     BETA_timecrit * s4 + \
     BETA_commuter * s5 + \
     BETA_AGE_50_65 * s6 + \
     BETA_AGE_65 * s7 + \
     BETA_IVT * ivt + \
     BETA_WT * wt

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [8]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "p2_001_general_MNL"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

BETA_AGE_50_65=	0.698
BETA_AGE_65=	1.98
BETA_CR32=	0.451
BETA_CR42=	2.78
BETA_CR43=	2.69
BETA_IVT=	0.0255
BETA_WT=	-0.298
BETA_commuter=	-0.166
BETA_timecrit=	-1.39
                   Value   Std err     t-test       p-value  Rob. Std err  \
BETA_AGE_50_65  0.697582  0.243664   2.862884  4.198042e-03      0.250785   
BETA_AGE_65     1.977972  0.241580   8.187646  2.220446e-16      0.251954   
BETA_CR32       0.451249  0.237337   1.901301  5.726259e-02      0.239882   
BETA_CR42       2.775394  0.249987  11.102172  0.000000e+00      0.249552   
BETA_CR43       2.688504  0.248856  10.803456  0.000000e+00      0.248277   
BETA_IVT        0.025473  0.004488   5.675749  1.380830e-08      0.004992   
BETA_WT        -0.297568  0.021836 -13.627206  0.000000e+00      0.021828   
BETA_commuter  -0.165960  0.143882  -1.153443  2.487285e-01      0.143811   
BETA_timecrit  -1.387579  0.110455 -12.562427  0.000000e+00      0.111137   

                Rob. t-test  Rob. p-value  
BETA_AGE_50_65     2

# 2. MNL - time-critical trips only

### Utility (dep1) = 0
### Utility (dep2) = [B_crowding_3-2] * s1 + [B_crowding_4-3] * s2 + [B_crowding_4-2] * s3 + [B_wt] * WT

In [9]:
# filter time-critical only records
df_analysis_timecrit = df_analysis[df_analysis['time_crit'] == 1]

In [10]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_analysis_timecrit)

In [11]:
# parameters to be estimated
ASC_FIRST = Beta('ASC_FIRST',0,None,None,1)
BETA_CR32 = Beta('BETA_CR32',0,None,None,0)
BETA_CR43 = Beta('BETA_CR43',0,None,None,0)
BETA_CR42 = Beta('BETA_CR42',0,None,None,0)
BETA_WT = Beta('BETA_WT',0,None,None,0)

# variables
s1 = Crowding_3_2 
s2 = Crowding_4_3
s3 = Crowding_4_2
wt = WT 

# utilities
V1 = ASC_FIRST

V2 = BETA_CR32 * s1 + \
     BETA_CR43 * s2 + \
     BETA_CR42 * s3 + \
     BETA_WT * wt

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [12]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "p2_002_MNL_timecrit"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

BETA_CR32=	-0.19
BETA_CR42=	1.91
BETA_CR43=	1.84
BETA_WT=	-0.303
              Value   Std err    t-test       p-value  Rob. Std err  \
BETA_CR32 -0.189619  0.271892 -0.697404  4.855500e-01      0.272909   
BETA_CR42  1.906381  0.257995  7.389212  1.476597e-13      0.257790   
BETA_CR43  1.835401  0.256520  7.155013  8.366641e-13      0.255936   
BETA_WT   -0.302536  0.031900 -9.483766  0.000000e+00      0.031850   

           Rob. t-test  Rob. p-value  
BETA_CR32    -0.694805  4.871777e-01  
BETA_CR42     7.395103  1.412204e-13  
BETA_CR43     7.171319  7.427392e-13  
BETA_WT      -9.498752  0.000000e+00  


# 3. MNL - non-time-critical trips only

### Utility (dep1) = 0
### Utility (dep2) = [B_crowding_3-2] * s1 + [B_crowding_4-3] * s2 + [B_crowding_4-2] * s3 + [B_wt] * WT

In [13]:
# filter non-time-critical only records
df_analysis_nontimecrit = df_analysis[df_analysis['time_crit'] == 0]

In [14]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_analysis_nontimecrit)

In [15]:
# parameters to be estimated
ASC_FIRST = Beta('ASC_FIRST',0,None,None,1)
BETA_CR32 = Beta('BETA_CR32',0,None,None,0)
BETA_CR43 = Beta('BETA_CR43',0,None,None,0)
BETA_CR42 = Beta('BETA_CR42',0,None,None,0)
BETA_WT = Beta('BETA_WT',0,None,None,0)

# variables
s1 = Crowding_3_2 
s2 = Crowding_4_3
s3 = Crowding_4_2
wt = WT 

# utilities
V1 = ASC_FIRST

V2 = BETA_CR32 * s1 + \
     BETA_CR43 * s2 + \
     BETA_CR42 * s3 + \
     BETA_WT * wt

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [16]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "p2_003_MNL_nontimecrit"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

BETA_CR32=	0.951
BETA_CR42=	3.06
BETA_CR43=	2.97
BETA_WT=	-0.256
              Value   Std err     t-test   p-value  Rob. Std err  Rob. t-test  \
BETA_CR32  0.951277  0.218555   4.352584  0.000013      0.217299     4.377739   
BETA_CR42  3.063424  0.255616  11.984493  0.000000      0.253482    12.085378   
BETA_CR43  2.971009  0.253453  11.722143  0.000000      0.253032    11.741648   
BETA_WT   -0.255532  0.027511  -9.288243  0.000000      0.027496    -9.293533   

           Rob. p-value  
BETA_CR32      0.000012  
BETA_CR42      0.000000  
BETA_CR43      0.000000  
BETA_WT        0.000000  


# 4. LC - 2 classes

##
## Utility function:
### Utility (dep1) = 0 (fixed reference value)
### Utility (dep2) = [B_crowding_3-2] * s1 + [B_crowding_4-3] * s2 + [B_wt] * WT
##
## Class membership function:
### CM =  [B_timecrit] * s3+ [B_commuter] * s4 + [B_age_50-65] * s5 + [B_age_65plus] * s6 + [B_jrt] * JRT

In [17]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_analysis)

In [18]:
# panel data - identifiable by pass_ID
database.panel("pass_ID")

In [19]:
# parameters to be estimated
ASC_FIRST_c1 = Beta('ASC_FIRST_c1',0,-10,10,1)
ASC_FIRST_c2 = Beta('ASC_FIRST_c2',0,-10,10,1)

#LC class 1
BETA_CR32_c1 = Beta('BETA_CR32_c1',1,-10,10,0)
BETA_CR43_c1 = Beta('BETA_CR43_c1',1,-10,10,0)
BETA_CR42_c1 = Beta('BETA_CR42_c1',1,-10,10,0)
BETA_WT_c1 = Beta('BETA_WT_c1',-0.1,-10,10,0)

#LC class 2
BETA_CR32_c2 = Beta('BETA_CR32_c2',1,-10,10,0)
BETA_CR43_c2 = Beta('BETA_CR43_c2',1,-10,10,0)
BETA_CR42_c2 = Beta('BETA_CR42_c2',1,-10,10,0)
BETA_WT_c2 = Beta('BETA_WT_c2',-0.1,-10,10,0)

In [20]:
# definition of variables
s1 = Crowding_3_2 
s2 = Crowding_4_3
s3 = Crowding_4_2
s4 = time_crit
s5 = commuter
s6 = age_50_65
s7 = age_65plus
ivt = JRT 
wt = WT

In [21]:
# utility function for LC class 1
V11 = ASC_FIRST_c1

V12 = BETA_CR32_c1 * s1 + \
      BETA_CR43_c1 * s2 + \
      BETA_CR42_c1 * s3 + \
      BETA_WT_c1 * wt

# Associate utility functions with the numbering of alternatives
V1 = {0: V11,
     1: V12}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [22]:
# utility function for LC class 2
V21 = ASC_FIRST_c2

V22 = BETA_CR32_c2 * s1 + \
      BETA_CR43_c2 * s2 + \
      BETA_CR42_c2 * s3 + \
      BETA_WT_c2 * wt

# Associate utility functions with the numbering of alternatives
V2 = {0: V21,
     1: V22}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [23]:
# class membership model

CL1_intercept = Beta('CL1_intercept',0,None,None,1)

CL2_intercept = Beta('CL2_intercept',0.001,-100,100,0)
CL2_timecrit = Beta('CL2_timecrit',-1,-100,100,0)
CL2_commuter = Beta('CL2_commuter',1, -100,100,0)
CL2_AGE_50_65 = Beta('CL2_AGE_50_65',1,-100,100,0)
CL2_AGE_65 = Beta('CL2_AGE_65',1,-100,100,0)
CL2_IVT = Beta('CL2_IVT',-0.1,-100,100,0)

CL1 = CL1_intercept
CL2 = CL2_intercept + \
      CL2_timecrit * s4 + \
      CL2_commuter * s5 + \
      CL2_AGE_50_65 * s6 + \
      CL2_AGE_65 * s7 + \
      CL2_IVT * ivt

probClass1 = models.logit({1:CL1,2:CL2},None,1)
probClass2 = models.logit({1:CL1,2:CL2},None,2)
#probClass2 = 1.0 / (1.0 + exp(CL2))
#probClass1 = 1.0 - probClass2

In [24]:
# Conditional to the random variables, likelihood if the individual is
# in class 1
#prob1 = models.logit(V1,av,output_choice)
prob1 = PanelLikelihoodTrajectory(models.logit(V1,av,output_choice))

# Conditional to the random variables, likelihood if the individual is
# in class 2
#prob2 = models.logit(V2,av,output_choice)
prob2 = PanelLikelihoodTrajectory(models.logit(V2,av,output_choice))

# Conditional to the random variables, likelihood for the individual.
probIndiv = probClass1 * prob1 + probClass2 * prob2

logprob = log(probIndiv)
# We integrate over the random variables using Monte-Carlo
#logprob = log(MonteCarlo(probIndiv))

In [25]:
# Define level of verbosity
import biogeme.messaging as msg
logger = msg.bioMessage()
#logger.setSilent()
#logger.setWarning()
logger.setGeneral()
#logger.setDetailed()

# Create the Biogeme object
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "p2_004_LC_2classes"

# Estimate the parameters. 
results = biogeme.estimate()
pandasResults = results.getEstimatedParameters()
print(pandasResults)

[18:49:21] < General >   Remove 2 unused variables from the database as only 11 are used.
[18:49:21] < General >   Log likelihood:  -1539.626
[18:49:21] < General >   Minimize with tol 9.999999999999999e-05
[18:49:21] < General >   Log likelihood:  -1539.626 Gradient norm:      2e+03
[18:49:21] < General >   Log likelihood:   -63473.1 Gradient norm:      7e+03
[18:49:21] < General >   Log likelihood:  -4445.233 Gradient norm:      6e+03
[18:49:21] < General >   Log likelihood:   -1459.01 Gradient norm:      7e+02
[18:49:21] < General >   Log likelihood:  -1449.876 Gradient norm:      4e+02
[18:49:21] < General >   Log likelihood:  -1421.656 Gradient norm:      5e+02
[18:49:21] < General >   Log likelihood:  -3957.322 Gradient norm:      5e+03
[18:49:21] < General >   Log likelihood:  -1384.425 Gradient norm:      2e+03
[18:49:21] < General >   Log likelihood:  -1321.612 Gradient norm:      2e+03
[18:49:21] < General >   Log likelihood:  -1202.093 Gradient norm:      1e+03
[18:49:21] < 

[18:49:22] < General >   Log likelihood:    -824.57 Gradient norm:          5
[18:49:22] < General >   Log likelihood:  -824.5681 Gradient norm:          5
[18:49:22] < General >   Log likelihood:  -824.5655 Gradient norm:          2
[18:49:22] < General >   Log likelihood:  -824.5614 Gradient norm:          4
[18:49:22] < General >   Log likelihood:  -824.5557 Gradient norm:      1e+01
[18:49:22] < General >   Log likelihood:   -824.549 Gradient norm:      1e+01
[18:49:22] < General >   Log likelihood:  -824.5494 Gradient norm:      1e+01
[18:49:22] < General >   Log likelihood:  -824.5463 Gradient norm:      1e+01
[18:49:22] < General >   Log likelihood:  -824.5434 Gradient norm:          7
[18:49:22] < General >   Log likelihood:  -824.5533 Gradient norm:      2e+01
[18:49:22] < General >   Log likelihood:  -824.5406 Gradient norm:          4
[18:49:22] < General >   Log likelihood:  -824.5382 Gradient norm:          3
[18:49:22] < General >   Log likelihood:  -824.5365 Gradient nor

[18:49:24] < General >   Log likelihood:  -824.3268 Gradient norm:        0.3
[18:49:24] < General >   Log likelihood:  -824.3267 Gradient norm:        0.2
[18:49:24] < General >   Log likelihood:  -824.3266 Gradient norm:        0.2
[18:49:24] < General >   Log likelihood:   -824.333 Gradient norm:      2e+01
[18:49:24] < General >   Log likelihood:  -824.3266 Gradient norm:        0.9
[18:49:24] < General >   Log likelihood:  -824.3265 Gradient norm:        0.5
[18:49:24] < General >   Log likelihood:  -824.3264 Gradient norm:        0.1
[18:49:24] < General >   Log likelihood:  -824.3263 Gradient norm:        0.2
[18:49:24] < General >   Log likelihood:  -824.3264 Gradient norm:        0.9
[18:49:24] < General >   Log likelihood:  -824.3263 Gradient norm:        0.2
[18:49:24] < General >   Log likelihood:  -824.3263 Gradient norm:       0.09
[18:49:24] < General >   Log likelihood:  -824.3263 Gradient norm:       0.09
[18:49:24] < General >   Log likelihood:  -824.3263 Gradient nor

[18:49:26] < General >   Log likelihood:  -824.3256 Gradient norm:          2
[18:49:26] < General >   Log likelihood:  -824.3255 Gradient norm:        0.3
[18:49:26] < General >   Log likelihood:  -824.3255 Gradient norm:        0.2
[18:49:26] < General >   Log likelihood:  -824.3255 Gradient norm:       0.07
[18:49:26] < General >   Log likelihood:  -824.3255 Gradient norm:       0.09
[18:49:26] < General >   Log likelihood:  -824.3255 Gradient norm:       0.07
[18:49:26] < General >   Log likelihood:  -824.3255 Gradient norm:        0.1
[18:49:26] < General >   Log likelihood:  -824.3255 Gradient norm:        0.4
[18:49:26] < General >   Log likelihood:  -824.3255 Gradient norm:       0.03
[18:49:26] < General >   Log likelihood:  -824.3255 Gradient norm:       0.04
[18:49:26] < General >   Log likelihood:  -824.3255 Gradient norm:       0.07
[18:49:26] < General >   Log likelihood:  -824.3255 Gradient norm:        0.1
[18:49:26] < General >   Log likelihood:  -824.3255 Gradient nor

[18:49:28] < General >   Log likelihood:  -824.3254 Gradient norm:       0.06
[18:49:28] < General >   Log likelihood:  -824.3254 Gradient norm:      0.007
[18:49:28] < General >   Log likelihood:  -824.3254 Gradient norm:       0.01
[18:49:28] < General >   Log likelihood:  -824.3254 Gradient norm:       0.02
[18:49:28] < General >   Log likelihood:  -824.3254 Gradient norm:       0.01
[18:49:28] < General >   Log likelihood:  -824.3254 Gradient norm:       0.01
[18:49:28] < General >   Log likelihood:  -824.3254 Gradient norm:       0.01
[18:49:28] < General >   Log likelihood:  -824.3254 Gradient norm:       0.02
[18:49:28] < General >   Log likelihood:  -824.3254 Gradient norm:        0.1
[18:49:28] < General >   Log likelihood:  -824.3254 Gradient norm:      0.009
[18:49:28] < General >   Log likelihood:  -824.3254 Gradient norm:      0.008
[18:49:28] < General >   Log likelihood:  -824.3254 Gradient norm:       0.01
[18:49:28] < General >   Log likelihood:  -824.3254 Gradient nor

[18:49:29] < General >   Log likelihood:  -824.3254 Gradient norm:     0.0005
[18:49:29] < General >   Log likelihood:  -824.3254 Gradient norm:       0.01
[18:49:29] < General >   Log likelihood:  -824.3254 Gradient norm:      0.002
[18:49:29] < General >   Log likelihood:  -824.3254 Gradient norm:     0.0008
[18:49:30] < General >   Log likelihood:  -824.3254 Gradient norm:     0.0006
[18:49:30] < General >   Log likelihood:  -824.3254 Gradient norm:     0.0005
[18:49:30] < General >   Log likelihood:  -824.3254 Gradient norm:       0.01
[18:49:30] < General >   Log likelihood:  -824.3254 Gradient norm:      0.001
[18:49:30] < General >   Log likelihood:  -824.3254 Gradient norm:     0.0008
[18:49:30] < General >   Log likelihood:  -824.3254 Gradient norm:     0.0005
[18:49:30] < General >   Log likelihood:  -824.3254 Gradient norm:     0.0006
[18:49:30] < General >   Log likelihood:  -824.3254 Gradient norm:     0.0009
[18:49:30] < General >   Log likelihood:  -824.3254 Gradient nor

# 5. LC - 3 classes

##
## Utility function:
### Utility (dep1) = 0 (fixed reference value)
### Utility (dep2) = [B_crowding_3-2] * s1 + [B_crowding_4-3] * s2 + [B_wt] * WT
##
## Class membership function:
### CM =  [B_timecrit] * s3+ [B_commuter] * s4 + [B_age_50-65] * s5 + [B_age_65plus] * s6 + [B_jrt] * JRT

In [26]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_analysis)

In [27]:
# panel data - identifiable by pass_ID
database.panel("pass_ID")

In [28]:
# parameters to be estimated
ASC_FIRST_c1 = Beta('ASC_FIRST_c1',0,-100,100,1)
ASC_FIRST_c2 = Beta('ASC_FIRST_c2',0,-100,100,1)
ASC_FIRST_c3 = Beta('ASC_FIRST_c3',0,-100,100,1)

#LC class 1
BETA_CR32_c1 = Beta('BETA_CR32_c1',1,-100,100,0)
BETA_CR43_c1 = Beta('BETA_CR43_c1',1,-100,100,0)
BETA_CR42_c1 = Beta('BETA_CR42_c1',1,-100,100,0)
BETA_WT_c1 = Beta('BETA_WT_c1',1,-100,100,0)

#LC class 2
BETA_CR32_c2 = Beta('BETA_CR32_c2',1,-100,100,0)
BETA_CR43_c2 = Beta('BETA_CR43_c2',1,-100,100,0)
BETA_CR42_c2 = Beta('BETA_CR42_c2',1,-100,100,0)
BETA_WT_c2 = Beta('BETA_WT_c2',1,-100,100,0)

#LC class 3
BETA_CR32_c3 = Beta('BETA_CR32_c3',1,-100,100,0)
BETA_CR43_c3 = Beta('BETA_CR43_c3',1,-100,100,0)
BETA_CR42_c3 = Beta('BETA_CR42_c3',1,-100,100,0)
BETA_WT_c3 = Beta('BETA_WT_c3',1,-100,100,0)

In [29]:
# definition of variables
s1 = Crowding_3_2 
s2 = Crowding_4_3
s3 = Crowding_4_2
s4 = time_crit
s5 = commuter
s6 = age_50_65
s7 = age_65plus
ivt = JRT 
wt = WT 

In [30]:
# utility function for LC class 1
V11 = ASC_FIRST_c1

V12 = BETA_CR32_c1 * s1 + \
      BETA_CR43_c1 * s2 + \
      BETA_CR42_c1 * s3 + \
      BETA_WT_c1 * wt

# Associate utility functions with the numbering of alternatives
V1 = {0: V11,
     1: V12}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [31]:
# utility function for LC class 2
V21 = ASC_FIRST_c2

V22 = BETA_CR32_c2 * s1 + \
      BETA_CR43_c2 * s2 + \
      BETA_CR42_c2 * s3 + \
      BETA_WT_c2 * wt

# Associate utility functions with the numbering of alternatives
V2 = {0: V21,
     1: V22}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [32]:
# utility function for LC class 3
V31 = ASC_FIRST_c3

V32 = BETA_CR32_c3 * s1 + \
      BETA_CR43_c3 * s2 + \
      BETA_CR42_c3 * s3 + \
      BETA_WT_c3 * wt

# Associate utility functions with the numbering of alternatives
V3 = {0: V31,
     1: V32}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [33]:
# class membership model

CL1_intercept = Beta('CL1_intercept',0,None,None,1)

CL2_intercept = Beta('CL2_intercept',0,-10,10,0)
CL2_timecrit = Beta('CL2_timecrit',-1,-10,10,0)
CL2_commuter = Beta('CL2_commuter',1, -10,10,0)
CL2_AGE_50_65 = Beta('CL2_AGE_50_65',1,-10,10,0)
CL2_AGE_65 = Beta('CL2_AGE_65',1,-10,10,0)
CL2_IVT = Beta('CL2_IVT',-0.1,-10,10,0)

CL3_intercept = Beta('CL3_intercept',0,-10,10,0)
CL3_timecrit = Beta('CL3_timecrit',-1,-10,10,0)
CL3_commuter = Beta('CL3_commuter',1, -10,10,0)
CL3_AGE_50_65 = Beta('CL3_AGE_50_65',1,-10,10,0)
CL3_AGE_65 = Beta('CL3_AGE_65',1,-10,10,0)
CL3_IVT = Beta('CL3_IVT',-0.1,-10,10,0)

CL1 = CL1_intercept

CL2 = CL2_intercept + \
      CL2_timecrit * s4 + \
      CL2_commuter * s5 + \
      CL2_AGE_50_65 * s6 + \
      CL2_AGE_65 * s7 + \
      CL2_IVT * ivt

CL3 = CL3_intercept + \
      CL3_timecrit * s4 + \
      CL3_commuter * s5 + \
      CL3_AGE_50_65 * s6 + \
      CL3_AGE_65 * s7 + \
      CL3_IVT * ivt

probClass1 = models.logit({1:CL1,2:CL2,3:CL3},None,1)
probClass2 = models.logit({1:CL1,2:CL2,3:CL3},None,2)
probClass3 = models.logit({1:CL1,2:CL2,3:CL3},None,3)

In [34]:
# Conditional to the random variables, likelihood if the individual is
# in class 1
prob1 = PanelLikelihoodTrajectory(models.logit(V1,av,output_choice))

# Conditional to the random variables, likelihood if the individual is
# in class 2
prob2 = PanelLikelihoodTrajectory(models.logit(V2,av,output_choice))

# Conditional to the random variables, likelihood if the individual is
# in class 3
prob3 = PanelLikelihoodTrajectory(models.logit(V3,av,output_choice))

# Conditional to the random variables, likelihood for the individual.
probIndiv = probClass1 * prob1 + probClass2 * prob2 + probClass3 * prob3

logprob = log(probIndiv)
# We integrate over the random variables using Monte-Carlo
#logprob = log(MonteCarlo(probIndiv))

In [35]:
# Define level of verbosity
import biogeme.messaging as msg
logger = msg.bioMessage()
#logger.setSilent()
#logger.setWarning()
logger.setGeneral()
#logger.setDetailed()

# Create the Biogeme object
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "p2_005_LC_3classes"

# Estimate the parameters. 
results = biogeme.estimate()
pandasResults = results.getEstimatedParameters()
print(pandasResults)

[18:49:31] < General >   Remove 2 unused variables from the database as only 11 are used.
[18:49:31] < General >   Log likelihood:   -10968.8
[18:49:31] < General >   Minimize with tol 9.999999999999999e-05
[18:49:31] < General >   Log likelihood:   -10968.8 Gradient norm:      7e+03
[18:49:31] < General >   Log likelihood: -1.797693e+308 Gradient norm:        inf
[18:49:31] < General >   Log likelihood:   -10968.8 Gradient norm:      7e+03
[18:49:31] < General >   Log likelihood:   -10968.8 Gradient norm:      7e+03
[18:49:32] < General >   Results saved in file p2_005_LC_3classes.html
[18:49:32] < General >   Results saved in file p2_005_LC_3classes.pickle
               Value   Std err    t-test   p-value  Rob. Std err  Rob. t-test  \
BETA_CR32_c1     1.0  7.451763  0.134196  0.893247   2350.011212     0.000426   
BETA_CR32_c2     1.0  7.451763  0.134196  0.893247   2350.011212     0.000426   
BETA_CR32_c3     1.0  7.451763  0.134196  0.893247   2350.011212     0.000426   
BETA_CR42

# 6. MXL (mixed logit) general model, normal distribution

## Utility (dep1) = 0 (fixed reference value)
## Utility (dep2) = booleans (s1, s2...) + travel times (wt, jrt)
### Utility (dep2) = = [B_crowding_3-2_RND] * s1 + [B_crowding_4-3_RND] * s2 + [B_crowding_4-2_RND] * s3 + [B_timecrit_RND] * s4 + [B_commuter_RND] * s5 + [B_age_50-65_RND] * s6 + [B_age_65plus_RND] * s7
###                        + [B_wt_RND] * WT + [B_jrt_RND] * JRT
###
## beta variables - RND with normal distribution

In [36]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_analysis)

In [37]:
# parameters to be estimated
# mean values
ASC_FIRST = Beta('ASC_FIRST',0,None,None,1)
BETA_CR32 = Beta('BETA_CR32',0,None,None,0)
BETA_CR43 = Beta('BETA_CR43',0,None,None,0)
BETA_CR42 = Beta('BETA_CR42',0,None,None,0)
BETA_timecrit = Beta('BETA_timecrit',0,None,None,0)
BETA_commuter = Beta('BETA_commuter',0, None,None,0)
BETA_AGE_50_65 = Beta('BETA_AGE_50_65',0,None,None,0)
BETA_AGE_65 = Beta('BETA_AGE_65',0,None,None,0)
BETA_IVT = Beta('BETA_IVT',0,None,None,0)
BETA_WT = Beta('BETA_WT',0,None,None,0)

# std distribution
BETA_CR32_S = Beta('BETA_CR32_S',0,None,None,0)
BETA_CR43_S = Beta('BETA_CR43_S',0,None,None,0)
BETA_CR42_S = Beta('BETA_CR42_S',0,None,None,0)
BETA_timecrit_S = Beta('BETA_timecrit_S',0,None,None,0)
BETA_commuter_S = Beta('BETA_commuter_S',0, None,None,0)
BETA_AGE_50_65_S = Beta('BETA_AGE_50_65_S',0,None,None,0)
BETA_AGE_65_S = Beta('BETA_AGE_65_S',0,None,None,0)
BETA_IVT_S = Beta('BETA_IVT_S',0,None,None,0)
BETA_WT_S = Beta('BETA_WT_S',0,None,None,0)

# Define a random parameter with a normal distribution, designed to be used
# for quasi Monte-Carlo simulation with Halton draws (base 5).

BETA_CR32_RND = BETA_CR32 + BETA_CR32_S * bioDraws('BETA_CR32_RND', 'NORMAL_HALTON5')
BETA_CR43_RND = BETA_CR43 + BETA_CR43_S * bioDraws('BETA_CR43_RND', 'NORMAL_HALTON5')
BETA_CR42_RND = BETA_CR42 + BETA_CR42_S * bioDraws('BETA_CR42_RND', 'NORMAL_HALTON5')
BETA_timecrit_RND = BETA_timecrit + BETA_timecrit_S * bioDraws('BETA_timecrit_RND', 'NORMAL_HALTON5')
BETA_commuter_RND = BETA_commuter + BETA_commuter_S * bioDraws('BETA_commuter_RND', 'NORMAL_HALTON5')
BETA_AGE_50_65_RND = BETA_AGE_50_65 + BETA_AGE_50_65_S * bioDraws('BETA_AGE_50_65_RND', 'NORMAL_HALTON5')
BETA_AGE_65_RND = BETA_AGE_65 + BETA_AGE_65_S * bioDraws('BETA_AGE_65_RND', 'NORMAL_HALTON5')
BETA_IVT_RND = BETA_IVT + BETA_IVT_S * bioDraws('BETA_IVT_RND', 'NORMAL_HALTON5')
BETA_WT_RND = BETA_WT + BETA_WT_S * bioDraws('BETA_WT_RND', 'NORMAL_HALTON5')

# variables
s1 = Crowding_3_2 
s2 = Crowding_4_3
s3 = Crowding_4_2
s4 = time_crit
s5 = commuter
s6 = age_50_65
s7 = age_65plus
ivt = JRT 
wt = WT 

# utilities
V1 = ASC_FIRST

V2 = BETA_CR32_RND * s1 + \
     BETA_CR43_RND * s2 + \
     BETA_CR42_RND * s3 + \
     BETA_timecrit_RND * s4 + \
     BETA_commuter_RND * s5 + \
     BETA_AGE_50_65_RND * s6 + \
     BETA_AGE_65_RND * s7 + \
     BETA_IVT_RND * ivt + \
     BETA_WT_RND * wt

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

In [38]:
# Conditional to B_TIME_RND, we have a logit model (called the kernel)
prob = models.logit(V,av,output_choice)

# integrate random variable betas using Monte-Carlo
logprob = log(MonteCarlo(prob))

# Define level of verbosity
import biogeme.messaging as msg
logger = msg.bioMessage()
#logger.setSilent()
#logger.setWarning()
logger.setGeneral()
#logger.setDetailed()

# Create the Biogeme object
biogeme = bio.BIOGEME(database,logprob,numberOfDraws=50)
biogeme.modelName = 'p2_006_general_MXL_normal'

# Estimate the parameters
results = biogeme.estimate()
pandasResults = results.getEstimatedParameters()
print(pandasResults)

[18:49:32] < General >   Remove 3 unused variables from the database as only 10 are used.
[18:49:40] < General >   Log likelihood:  -1555.422
[18:49:40] < General >   Minimize with tol 9.999999999999999e-05
[18:49:40] < General >   Log likelihood:  -1555.422 Gradient norm:      1e+03
[18:49:41] < General >   Log likelihood:  -12268.99 Gradient norm:      3e+04
[18:49:42] < General >   Log likelihood:  -1555.971 Gradient norm:      5e+03
[18:49:44] < General >   Log likelihood:  -1544.223 Gradient norm:      2e+03
[18:49:44] < General >   Log likelihood:   -1528.64 Gradient norm:      2e+03
[18:49:45] < General >   Log likelihood:  -1466.783 Gradient norm:      3e+02
[18:49:46] < General >   Log likelihood:  -1464.082 Gradient norm:      3e+02
[18:49:47] < General >   Log likelihood:  -1450.362 Gradient norm:      5e+02
[18:49:48] < General >   Log likelihood:  -1398.197 Gradient norm:      8e+02
[18:49:49] < General >   Log likelihood:  -1370.611 Gradient norm:      6e+02
[18:49:50] < 

[18:51:27] < General >   Log likelihood:  -1110.412 Gradient norm:      5e+02
[18:51:28] < General >   Log likelihood:  -1107.039 Gradient norm:      8e+01
[18:51:29] < General >   Log likelihood:  -1106.769 Gradient norm:      3e+01
[18:51:30] < General >   Log likelihood:  -1106.637 Gradient norm:      6e+01
[18:51:31] < General >   Log likelihood:    -1106.3 Gradient norm:      6e+01
[18:51:32] < General >   Log likelihood:   -1105.78 Gradient norm:      2e+01
[18:51:33] < General >   Log likelihood:  -1106.327 Gradient norm:      2e+02
[18:51:34] < General >   Log likelihood:  -1105.732 Gradient norm:      4e+01
[18:51:36] < General >   Log likelihood:  -1105.646 Gradient norm:      3e+01
[18:51:37] < General >   Log likelihood:  -1105.498 Gradient norm:      1e+02
[18:51:38] < General >   Log likelihood:  -1105.798 Gradient norm:      3e+02
[18:51:39] < General >   Log likelihood:   -1105.43 Gradient norm:      2e+01
[18:51:40] < General >   Log likelihood:  -1105.402 Gradient nor

[18:53:21] < General >   Log likelihood:  -1102.889 Gradient norm:          8
[18:53:22] < General >   Log likelihood:  -1102.878 Gradient norm:          9
[18:53:23] < General >   Log likelihood:  -1103.278 Gradient norm:      2e+02
[18:53:24] < General >   Log likelihood:  -1102.874 Gradient norm:      1e+01
[18:53:25] < General >   Log likelihood:  -1102.857 Gradient norm:      1e+01
[18:53:26] < General >   Log likelihood:  -1102.786 Gradient norm:      1e+01
[18:53:27] < General >   Log likelihood:  -1102.749 Gradient norm:      2e+01
[18:53:29] < General >   Log likelihood:  -1102.742 Gradient norm:      3e+01
[18:53:30] < General >   Log likelihood:  -1102.708 Gradient norm:          9
[18:53:31] < General >   Log likelihood:  -1102.693 Gradient norm:          8
[18:53:32] < General >   Log likelihood:   -1102.65 Gradient norm:      5e+01
[18:53:33] < General >   Log likelihood:  -1102.605 Gradient norm:      4e+01
[18:53:34] < General >   Log likelihood:   -1102.55 Gradient nor

[18:55:18] < General >   Log likelihood:  -1100.989 Gradient norm:          6
[18:55:19] < General >   Log likelihood:  -1100.988 Gradient norm:          6
[18:55:21] < General >   Log likelihood:  -1103.581 Gradient norm:      4e+02
[18:55:22] < General >   Log likelihood:  -1100.987 Gradient norm:      1e+01
[18:55:23] < General >   Log likelihood:  -1100.986 Gradient norm:          5
[18:55:24] < General >   Log likelihood:  -1100.985 Gradient norm:          3
[18:55:26] < General >   Log likelihood:  -1100.984 Gradient norm:          3
[18:55:27] < General >   Log likelihood:  -1100.983 Gradient norm:          4
[18:55:28] < General >   Log likelihood:  -1101.505 Gradient norm:      2e+02
[18:55:29] < General >   Log likelihood:  -1100.983 Gradient norm:          2
[18:55:31] < General >   Log likelihood:  -1100.981 Gradient norm:          2
[18:55:32] < General >   Log likelihood:  -1100.985 Gradient norm:      2e+01
[18:55:33] < General >   Log likelihood:  -1100.981 Gradient nor

[18:57:26] < General >   Log likelihood:  -1100.809 Gradient norm:          4
[18:57:28] < General >   Log likelihood:  -1100.808 Gradient norm:          2
[18:57:29] < General >   Log likelihood:  -1100.808 Gradient norm:          2
[18:57:30] < General >   Log likelihood:  -1100.809 Gradient norm:      1e+01
[18:57:31] < General >   Log likelihood:  -1100.807 Gradient norm:          6
[18:57:32] < General >   Log likelihood:  -1100.805 Gradient norm:          8
[18:57:33] < General >   Log likelihood:  -1100.802 Gradient norm:          3
[18:57:34] < General >   Log likelihood:  -1100.799 Gradient norm:          2
[18:57:35] < General >   Log likelihood:    -1100.8 Gradient norm:      2e+01
[18:57:36] < General >   Log likelihood:  -1100.799 Gradient norm:          7
[18:57:37] < General >   Log likelihood:  -1100.799 Gradient norm:          3
[18:57:38] < General >   Log likelihood:  -1100.798 Gradient norm:          2
[18:57:39] < General >   Log likelihood:  -1100.798 Gradient nor

...

[19:02:56] < General >   Log likelihood:  -1100.708 Gradient norm:        0.6
[19:02:57] < General >   Log likelihood:  -1100.708 Gradient norm:        0.5
[19:02:58] < General >   Log likelihood:  -1100.708 Gradient norm:        0.3
[19:02:59] < General >   Log likelihood:  -1100.708 Gradient norm:       0.09
[19:03:00] < General >   Log likelihood:  -1100.708 Gradient norm:        0.1
[19:03:01] < General >   Log likelihood:  -1100.708 Gradient norm:        0.2
[19:03:02] < General >   Log likelihood:  -1100.708 Gradient norm:       0.08
[19:03:03] < General >   Log likelihood:  -1100.708 Gradient norm:        0.1
[19:03:05] < General >   Log likelihood:  -1100.708 Gradient norm:        0.8
[19:03:06] < General >   Log likelihood:  -1100.708 Gradient norm:        0.5
[19:03:07] < General >   Log likelihood:  -1100.708 Gradient norm:        0.1
[19:03:09] < General >   Log likelihood:  -1100.708 Gradient norm:        0.2
[19:03:10] < General >   Log likelihood:  -1100.708 Gradient nor

[19:04:42] < General >   Log likelihood:  -1100.708 Gradient norm:       0.09
[19:04:43] < General >   Log likelihood:  -1100.708 Gradient norm:       0.03
[19:04:44] < General >   Log likelihood:  -1100.708 Gradient norm:       0.04
[19:04:45] < General >   Log likelihood:  -1100.708 Gradient norm:        0.2
[19:04:46] < General >   Log likelihood:  -1100.708 Gradient norm:        0.1
[19:04:47] < General >   Log likelihood:  -1100.708 Gradient norm:       0.07
[19:04:48] < General >   Log likelihood:  -1100.708 Gradient norm:        0.1
[19:04:48] < General >   Log likelihood:  -1100.708 Gradient norm:       0.03
[19:04:49] < General >   Log likelihood:  -1100.708 Gradient norm:       0.02
[19:04:50] < General >   Log likelihood:  -1100.708 Gradient norm:       0.04
[19:04:51] < General >   Log likelihood:  -1100.708 Gradient norm:       0.08
[19:04:52] < General >   Log likelihood:  -1100.708 Gradient norm:       0.03
[19:04:53] < General >   Log likelihood:  -1100.708 Gradient nor

[19:06:22] < General >   Log likelihood:  -1100.708 Gradient norm:       0.05
[19:06:23] < General >   Log likelihood:  -1100.708 Gradient norm:       0.03
[19:06:23] < General >   Log likelihood:  -1100.708 Gradient norm:       0.08
[19:06:24] < General >   Log likelihood:  -1100.708 Gradient norm:       0.02
[19:06:25] < General >   Log likelihood:  -1100.708 Gradient norm:       0.01
[19:06:26] < General >   Log likelihood:  -1100.708 Gradient norm:      0.006
[19:06:27] < General >   Log likelihood:  -1100.708 Gradient norm:       0.01
[19:06:28] < General >   Log likelihood:  -1100.708 Gradient norm:       0.08
[19:06:29] < General >   Log likelihood:  -1100.708 Gradient norm:       0.02
[19:06:30] < General >   Log likelihood:  -1100.708 Gradient norm:      0.009
[19:06:31] < General >   Log likelihood:  -1100.708 Gradient norm:      0.006
[19:06:32] < General >   Log likelihood:  -1100.708 Gradient norm:      0.008
[19:06:33] < General >   Log likelihood:  -1100.708 Gradient nor

[19:08:01] < General >   Log likelihood:  -1100.708 Gradient norm:       0.01
[19:08:02] < General >   Log likelihood:  -1100.708 Gradient norm:      0.006
[19:08:03] < General >   Log likelihood:  -1100.708 Gradient norm:       0.04
[19:08:04] < General >   Log likelihood:  -1100.708 Gradient norm:       0.01
[19:08:05] < General >   Log likelihood:  -1100.708 Gradient norm:      0.009
[19:08:06] < General >   Log likelihood:  -1100.708 Gradient norm:       0.03
[19:08:07] < General >   Log likelihood:  -1100.708 Gradient norm:      0.009
[19:08:08] < General >   Log likelihood:  -1100.708 Gradient norm:      0.003
[19:08:09] < General >   Log likelihood:  -1100.708 Gradient norm:      0.003
[19:08:10] < General >   Log likelihood:  -1100.708 Gradient norm:      0.006
[19:08:11] < General >   Log likelihood:  -1100.708 Gradient norm:        0.1
[19:08:11] < General >   Log likelihood:  -1100.708 Gradient norm:       0.04
[19:08:12] < General >   Log likelihood:  -1100.708 Gradient nor

[19:09:42] < General >   Log likelihood:  -1100.708 Gradient norm:     0.0006
[19:09:43] < General >   Log likelihood:  -1100.708 Gradient norm:      0.005
[19:09:44] < General >   Log likelihood:  -1100.708 Gradient norm:      0.001
[19:09:45] < General >   Log likelihood:  -1100.708 Gradient norm:     0.0004
[19:09:46] < General >   Log likelihood:  -1100.708 Gradient norm:     0.0004
[19:09:47] < General >   Log likelihood:  -1100.708 Gradient norm:     0.0008
[19:09:48] < General >   Log likelihood:  -1100.708 Gradient norm:      0.001
[19:09:49] < General >   Log likelihood:  -1100.708 Gradient norm:      0.001
[19:09:50] < General >   Log likelihood:  -1100.708 Gradient norm:      0.008
[19:09:51] < General >   Log likelihood:  -1100.708 Gradient norm:      0.001
[19:09:51] < General >   Log likelihood:  -1100.708 Gradient norm:      0.001
[19:09:52] < General >   Log likelihood:  -1100.708 Gradient norm:     0.0005
[19:09:53] < General >   Log likelihood:  -1100.708 Gradient nor